# 04 - Comparación de Modelos y Almacenamiento en Base de Datos

In [1]:
import os
import sys
import pandas as pd

# 1. Añadir la carpeta 'src' al path para importar los módulos
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
    
from src import model_training
from src import db_save_results

## 2. Configurar la Base de Datos
Aseguramos que la base de datos y todas las tablas estén creadas.

In [2]:
db_save_results.setup_database()

Base de datos configurada en: ..\models\propiedades.db


## 3. Cargar Datos y Guardar (Datos de Entrada)
Cargamos el dataset procesado y filtrado (Venta o Alquiler) y lo almacenamos en la base de datos.

In [3]:
try:
    df = pd.read_csv('../data/processed/properati_features.csv')
    print("Datos procesados ('properati_features.csv') cargados.")
except FileNotFoundError:
    print("Error: No se encontró 'properati_features.csv'. Asegúrate de ejecutar el notebook 03.")
    df = pd.DataFrame()

# --- Determinar dinámicamente el target ---
TARGET_COLUMN = None
if 'price_usd' in df.columns:
    TARGET_COLUMN = 'price_usd'
elif 'price_ars' in df.columns:
    TARGET_COLUMN = 'price_ars'
else:
    print("ADVERTENCIA: No se encontró una columna de precio ('price_usd' o 'price_ars').")

if TARGET_COLUMN:
    print(f"Columna objetivo detectada: {TARGET_COLUMN}")
    # Renombramos el target a 'price_usd' para que coincida con el SQL (opcional, pero buena práctica)
    # Tu tabla SQL espera 'price_usd', así que hacemos este cambio
    df_to_save = df.rename(columns={TARGET_COLUMN: 'price_usd'})
    db_save_results.insert_input_data(df_to_save, 'datos_propiedades')


Datos procesados ('properati_features.csv') cargados.
Columna objetivo detectada: price_usd
Datos de entrada guardados en la tabla 'datos_propiedades'.


## 4. Entrenar Modelos y Obtener Resultados
Ejecutamos la función de entrenamiento, pasándole el nombre de la columna objetivo que detectamos.

In [4]:
if TARGET_COLUMN and not df.empty:
    # Ejecutamos el entrenamiento y capturamos los resultados
    models, metrics_df, config_list, X_test, y_test = model_training.train_models(
        df,
        target=TARGET_COLUMN, 
        out_dir='../models'
    )

    print("Métricas obtenidas del entrenamiento:")
    print(metrics_df.head())
else:
    print("No se puede entrenar el modelo. Verifica que el DataFrame 'df' no esté vacío y tenga una columna 'target'.")

Entrenando Linear Regression...
Entrenando Random Forest (con GridSearchCV)...
Entrenamiento completado.
Métricas obtenidas del entrenamiento:
   modelo         rmse           mae        r2
0  linear  80336.72449  45948.240782  0.678285
1      rf  59787.79083  32168.849976  0.821816


## 5. Almacenar Resultados en la Base de Datos
Guardamos las métricas y las configuraciones de los modelos en la base de datos.

In [5]:
if TARGET_COLUMN and 'metrics_df' in locals():
    # Guardar Métricas (RMSE, MAE, R2)
    db_save_results.insert_metrics(metrics_df)

    # Guardar Configuración (Hiperparámetros)
    db_save_results.insert_model_config(config_list)

    # Guardamos el CSV que necesita el notebook 05_visualizations
    metrics_df.to_csv('../data/processed/metrics_comparison.csv', index=False)

    print("¡Éxito! Métricas y configuraciones guardadas en la base de datos y en 'data/processed'.")
else:
    print("No hay resultados para guardar. Verifica que el entrenamiento se haya completado.")

Métricas del modelo guardadas en la base de datos.
Configuración de modelos guardada en la base de datos.
¡Éxito! Métricas y configuraciones guardadas en la base de datos y en 'data/processed'.
